<a href="https://colab.research.google.com/github/ucfilho/SwarmCantStopNow/blob/main/Conference_2021/ABCOptmi_simple_benchmark_sphere_D_30_pop_50_ite_500_RTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import randint
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
!git clone https://github.com/ucfilho/Raianars_Routines #clonar do Github
%cd Raianars_Routines

fatal: destination path 'Raianars_Routines' already exists and is not an empty directory.
/content/Raianars_Routines


In [3]:
############################### Função
import FunctionsNew
############################### Função
dim=30 # 
fchoice='Sphere'
function=FunctionsNew.Sphere
MAX = []
MIN = []

for i in  range(dim):
  MAX.append(5.12)
  MIN.append(-5.12)


In [4]:
def CalcFit(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+fabs(fun)
    return result

In [5]:
def BestSource(GlobMin,GlobPars,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobMin):
      GlobMin=f[i]
      for j in range(D):
        GlobPars[j]=Foods[i,j]
  return GlobMin,GlobPars,Foods

In [6]:
def init(index,Foods,trial,f,fitness,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(MAX[j]-MIN[j])+MIN[j]
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalcFit(f[index])
  trial[index]=0
  
  return Foods,trial,f,fitness

In [7]:
def initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    Foods,trial,f,fitness=init(i,Foods,trial,f,fitness,MIN,MAX) 
  GlobMin=f[0]
  for i in range(D):
    GlobPars[i]=Foods[0,i]
  return  f,Foods,GlobMin,GlobPars


In [8]:
def EmployedBees(trial,Foods,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  NP=FoodNumber
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    par2chan= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]

    r = np.random.random()
    A=Foods[i,par2chan]
    B=Foods[neighbour,par2chan]
    solution[par2chan]=Foods[i,par2chan]+(A-B)*(r-0.5)*2;
    if (solution[par2chan]<MIN[par2chan]):
      solution[par2chan]=MIN[par2chan]
    if (solution[par2chan]>MAX[par2chan]):
      solution[par2chan]=MAX[par2chan]
    ObjValSol=function(solution)
    FitnessSol=CalcFit(ObjValSol)
    
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 
      
  return trial,Foods

In [9]:
def CalcProb(fitness,prob):
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1
  return fitness,prob


In [10]:
def OnlookerBees(trial,Foods,MIN,MAX):
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      par2chan=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,par2chan]
      B=Foods[neighbour,par2chan]
      solution[par2chan]=A+(A-B)*(r-0.5)*2

      if (solution[par2chan]<MIN[par2chan]):
              solution[par2chan]=MIN[par2chan]
      if (solution[par2chan]>MAX[par2chan]):
              solution[par2chan]=MAX[par2chan]
      ObjValSol=function(solution)
      FitnessSol=CalcFit(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0
  return trial,Foods


In [11]:
def ScoutBees(fitness,f,Foods,trial,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  max_trial=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[max_trial]):
      max_trial=i
  if(trial[max_trial]>=limit):
    Foods,trial,f,fitness=init(max_trial,Foods,trial,f,fitness,MIN,MAX)
  return trial,Foods


In [12]:
'''/*Main program of the ABC algorithm*/'''

NPAR=50 #Numero de fontes de comida
ITE=500 #ITERACOES (maxCycle)

PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS
limit=1000 # quantas vezes obtem resposta identica antes de encerrar 
runtime=3 # quantas vezes vai rodar para tirar a media


In [13]:

f=np.zeros(NPAR) 
Foods=np.zeros((NPAR,PAR)) 
solution=np.zeros(PAR)
fitness=np.zeros(NPAR)
trial=np.zeros(NPAR)
prob=np.zeros(NPAR)
GlobPars=np.zeros(PAR)
GlobMins=np.zeros(runtime)
GlobMin=1e99


In [14]:
# rule half-half
N_2 = int(NPAR/1)

In [15]:
f1=f
Foods1=Foods
solution1= np.zeros(PAR)
fitness1=fitness
trial1=trial
prob1=prob
GlobPars1=np.zeros(PAR)
GlobMins1=np.zeros(runtime)
GlobMin1=1e99

In [16]:
initial_time = time.time()

for run in range(runtime):
  f1,Foods1,GlobMin1,GlobPars1= initial(fitness1,trial1,f1,Foods1,GlobMin1,GlobPars1,MIN,MAX)
  GlobMin1,GlobPars1,Foods1=BestSource(GlobMin1,GlobPars1,Foods1)



  for iter in range(ITE):
    trial1,Foods1=EmployedBees(trial1,Foods1,MIN,MAX);


    fitness1,prob1=CalcProb(fitness1,prob1);

    trial1,Foods1=OnlookerBees(trial1,Foods1,MIN,MAX);

    GlobMin1,GlobPars1,Foods1=BestSource(GlobMin1,GlobPars1,Foods1);

    trial1,Foods1=ScoutBees(fitness1,f1,Foods1,trial1,MIN,MAX)




  '''
  for j in range(PAR):
    print("GlobalParam[",j+1,"]:", GlobPars1[j])

  '''
  current_time = time.time() -initial_time   
  print("run",run+1,"time =",current_time ,"GBest=",GlobMin1," \n");
  GlobMins1[run]=GlobMin1

mean=np.average(GlobMins1)
sd_mean = np.std(GlobMins1)
best_ans = GlobMins1.min()
print('====dim:',dim,'===== pop:',NPAR,'===== ITE:',ITE,'====')
print("Means of",runtime,"runs:",mean,"\n")
print("Best ans of",runtime,"runs:",best_ans,"\n")
print("Deviation of",runtime,"runs:",sd_mean,"\n")

run 1 time = 1.9565973281860352 GBest= 1.8798311681037465e-06  

run 2 time = 3.8450963497161865 GBest= 1.7610817905559507e-06  

run 3 time = 5.717714786529541 GBest= 3.1807439485139974e-06  

====dim: 30 ===== pop: 50 ===== ITE: 500 ====
Means of 3 runs: 2.273885635724565e-06 

Best ans of 3 runs: 1.7610817905559507e-06 

Deviation of 3 runs: 6.430756063758338e-07 



In [17]:
GlobPars1

array([-4.74784771e-06, -2.02925940e-04,  3.76487413e-04, -8.16923497e-04,
       -3.73263062e-05, -1.12312241e-05,  9.48609496e-06, -2.18913766e-05,
        1.17933349e-03,  2.76607423e-04, -3.87920468e-06, -4.97316169e-05,
        2.98079909e-04,  1.34536961e-05, -5.09607424e-05,  3.45723115e-04,
       -7.24459233e-07, -4.66660069e-07, -1.37084913e-04,  7.79739032e-05,
        3.53753346e-05,  1.10829387e-04, -3.26242612e-05, -8.02733562e-05,
       -8.34280075e-06,  2.83396322e-04,  9.28096749e-06,  3.56697863e-07,
        3.02346077e-04, -6.55413917e-04])